In [2]:
import xarray
import pandas as pd 
import numpy as np

In [2]:
xa = xarray.open_dataset("tmax.nc", decode_times=False)
tmax = xa.to_dataframe().reset_index()
tmax.rename(columns={"X": "lon", "Y": "lat", "T": "months"}, inplace=True)
tmax.head()

,lon,months,lat,tmax
0,-92.525002,240.5,13.474999,NaN
1,-92.525002,240.5,13.525000,NaN
2,-92.525002,240.5,13.575000,NaN
3,-92.525002,240.5,13.624999,NaN
4,-92.525002,240.5,13.674999,NaN


In [3]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [6]:
def process_date(months):
    return dt.datetime(1960, 1, 1) + relativedelta(months=int(months))

tmax["date"] = tmax.months.map(process_date)

In [7]:
tmax["mes"] = tmax.date.map(lambda x: x.month)
tmax["anio"] = tmax.date.map(lambda x: x.year)
tmax = tmax[tmax.anio > 2001]
tmax.head()

,lon,months,lat,tmax,date,mes,anio
26928,-92.525002,504.5,13.474999,NaN,2002-01-01,1,2002
26929,-92.525002,504.5,13.525000,NaN,2002-01-01,1,2002
26930,-92.525002,504.5,13.575000,NaN,2002-01-01,1,2002
26931,-92.525002,504.5,13.624999,NaN,2002-01-01,1,2002
26932,-92.525002,504.5,13.674999,NaN,2002-01-01,1,2002


In [8]:
ndvi = pd.read_csv("ndvi-lluvia-tmin.csv")
ndvi.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat,lluvia_code,precipitacion,tmin_code,tmin
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487


In [9]:
adm2 = ndvi.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]
adm2.head()

/var/folders/sg/bv36hz0d5z36h52fqpd37kn80000gn/T/ipykernel_32878/2444717534.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  adm2 = ndvi.groupby("ADM2_PCODE").mean().reset_index()[["ADM2_PCODE", "lat", "lon"]]


,ADM2_PCODE,lat,lon
0,GT0100,14.435029,-90.548878
1,GT0101,14.635431,-90.479418
2,GT0102,14.564392,-90.465079
3,GT0103,14.554316,-90.346363
4,GT0104,14.793193,-90.372463


In [10]:
coord = tmax[["lon", "lat"]].drop_duplicates()
coord["tmax_code"] = range(len(coord))
coord.head()

,lon,lat,tmax_code
26928,-92.525002,13.474999,0
26929,-92.525002,13.525000,1
26930,-92.525002,13.575000,2
26931,-92.525002,13.624999,3
26932,-92.525002,13.674999,4


In [11]:
def lluvia_code(row):
    dist = np.sqrt((coord.lon - row.lon)**2 + (coord.lat - row.lat)**2)
    return int(coord.iloc[dist.argmin()]["tmax_code"])


adm2["tmax_code"] = adm2.apply(lluvia_code, axis=1)
adm2.head()

,ADM2_PCODE,lat,lon,tmax_code
0,GT0100,14.435029,-90.548878,4099
1,GT0101,14.635431,-90.479418,4205
2,GT0102,14.564392,-90.465079,4204
3,GT0103,14.554316,-90.346363,4510
4,GT0104,14.793193,-90.372463,4412


In [12]:
tmax = tmax.merge(coord, "left", on=["lat", "lon"])
ndvi = ndvi.merge(adm2[["ADM2_PCODE", "tmax_code"]], "left", on="ADM2_PCODE")
ndvi = ndvi.merge(tmax[["tmax_code", "mes", "anio", "tmax"]], "left", on=["tmax_code", "mes", "anio"])
ndvi.head()

,date,adm2_id,ADM2_PCODE,n_pixels,vim,vim_avg,viq,mes,anio,dia,departamento,municipio,lon,lat,lluvia_code,precipitacion,tmin_code,tmin,tmax_code,tmax
0,2002-07-01,65298,GT1802,5.0,0.7442,0.7380,100.7902,7,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774,6880,30.219288
1,2002-07-11,65298,GT1802,5.0,0.7446,0.7394,100.6574,7,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774,6880,30.219288
2,2002-07-21,65298,GT1802,5.0,0.7452,0.7423,100.3636,7,2002,21,Izabal,Lívingston,-89.151105,15.777356,6645,699.3845,6880,21.887774,6880,30.219288
3,2002-08-01,65298,GT1802,5.0,0.7461,0.7456,100.0641,8,2002,1,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487,6880,33.259544
4,2002-08-11,65298,GT1802,5.0,0.7476,0.7490,99.8260,8,2002,11,Izabal,Lívingston,-89.151105,15.777356,6645,471.7211,6880,21.571487,6880,33.259544


In [14]:
ndvi = ndvi[ndvi.anio <= 2023]
ndvi.drop(["lluvia_code", "tmin_code", "tmax_code", "adm2_id"], inplace=True, axis=1)
ndvi.to_csv("ndvi-lluvia-tmin-tmax.csv", index=False)

In [17]:
ndvi = ndvi.groupby(["anio", "mes", "ADM2_PCODE", "departamento", "municipio"]).mean().reset_index()
ndvi.drop("vim_avg", axis=1, inplace=True)
ndvi.head()

,anio,mes,ADM2_PCODE,departamento,municipio,n_pixels,vim,viq,dia,lon,lat,precipitacion,tmin,tmax
0,2002,7,GT0100,Guatemala,Lago De Amatitlan,1.0,0.686067,100.902100,11.0,-90.548878,14.435029,169.31514,16.139826,25.118555
1,2002,7,GT0101,Guatemala,Guatemala,6.0,0.586433,101.562333,11.0,-90.479418,14.635431,191.59773,15.752864,25.048338
2,2002,7,GT0102,Guatemala,Santa Catarina Pinula,2.0,0.707967,101.330167,11.0,-90.465079,14.564392,186.14903,14.537229,23.773886
3,2002,7,GT0103,Guatemala,San José Pinula,5.0,0.796333,100.334500,11.0,-90.346363,14.554316,199.11770,11.996284,20.399481
4,2002,7,GT0104,Guatemala,San José del Golfo,2.0,0.774200,98.992867,11.0,-90.372463,14.793193,135.58841,19.454800,28.777624


In [20]:
ndvi["date"] = ndvi.apply(lambda row: dt.datetime(row.anio, row.mes, int(row.dia)), axis=1)

In [21]:
ndvi.head()

,anio,mes,ADM2_PCODE,departamento,municipio,n_pixels,vim,viq,dia,lon,lat,precipitacion,tmin,tmax,date
0,2002,7,GT0100,Guatemala,Lago De Amatitlan,1.0,0.686067,100.902100,11.0,-90.548878,14.435029,169.31514,16.139826,25.118555,2002-07-11
1,2002,7,GT0101,Guatemala,Guatemala,6.0,0.586433,101.562333,11.0,-90.479418,14.635431,191.59773,15.752864,25.048338,2002-07-11
2,2002,7,GT0102,Guatemala,Santa Catarina Pinula,2.0,0.707967,101.330167,11.0,-90.465079,14.564392,186.14903,14.537229,23.773886,2002-07-11
3,2002,7,GT0103,Guatemala,San José Pinula,5.0,0.796333,100.334500,11.0,-90.346363,14.554316,199.11770,11.996284,20.399481,2002-07-11
4,2002,7,GT0104,Guatemala,San José del Golfo,2.0,0.774200,98.992867,11.0,-90.372463,14.793193,135.58841,19.454800,28.777624,2002-07-11


In [23]:
ndvi.drop("dia", axis=1, inplace=True)
ndvi.to_csv("ndvi-lluvia-tmin-tmax.csv", index=False)

In [3]:
ndvi = pd.read_csv("ndvi-lluvia-tmin-tmax.csv")
ndvi["tdelta"] = ndvi.tmax - ndvi.tmin
ndvi = ndvi[ndvi.tdelta >= 0]
ndvi.to_csv("ndvi-lluvia-tmin-tmax.csv", index=False)
ndvi.head()

,anio,mes,ADM2_PCODE,departamento,municipio,n_pixels,vim,viq,lon,lat,precipitacion,tmin,tmax,date,tdelta
0,2002,7,GT0100,Guatemala,Lago De Amatitlan,1.0,0.686067,100.902100,-90.548878,14.435029,169.31514,16.139826,25.118555,2002-07-11,8.978729
1,2002,7,GT0101,Guatemala,Guatemala,6.0,0.586433,101.562333,-90.479418,14.635431,191.59773,15.752864,25.048338,2002-07-11,9.295474
2,2002,7,GT0102,Guatemala,Santa Catarina Pinula,2.0,0.707967,101.330167,-90.465079,14.564392,186.14903,14.537229,23.773886,2002-07-11,9.236657
3,2002,7,GT0103,Guatemala,San José Pinula,5.0,0.796333,100.334500,-90.346363,14.554316,199.11770,11.996284,20.399480,2002-07-11,8.403196
4,2002,7,GT0104,Guatemala,San José del Golfo,2.0,0.774200,98.992867,-90.372463,14.793193,135.58841,19.454800,28.777624,2002-07-11,9.322824
